In [1]:
from __future__ import print_function
import keras
from keras.datasets import cifar10
from keras.models import Model, Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D, Input, Dense
from keras import backend as K
from keras.preprocessing.image import ImageDataGenerator
import random
import matplotlib.pyplot as plt
import numpy as np

%matplotlib inline

Using TensorFlow backend.


In [2]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

In [3]:
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')
input_shape = (32, 32, 3)

x_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples


In [4]:
num_classes = 10
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

In [5]:
datagen = ImageDataGenerator(
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True
    )
datagen.fit(x_train)

In [6]:
def set_up_model():
    keras.backend.clear_session()
    inputs = Input(shape=input_shape)
    x = Conv2D(32, kernel_size=(3,3), strides=(1, 1), activation='relu')(inputs)
    x = Conv2D(32, kernel_size=(3,3), strides=(1, 1), activation='relu')(x)
    x = Dropout(0.25)(x)
    x = MaxPooling2D(pool_size=(2,2), strides=(2,2))(x)
    x = Conv2D(64, kernel_size=(3,3), strides=(1, 1), activation='relu')(x)
    x = Conv2D(64, kernel_size=(3,3), strides=(1, 1), activation='relu')(x)
    x = MaxPooling2D(pool_size=(2,2), strides=(2,2))(x)
    x = Dropout(0.25)(x)
    x = Flatten()(x)
    x = Dense(512, activation='relu')(x)
    x = Dropout(0.5)(x)
    predictions = Dense(10, activation='softmax')(x)
    model = Model(inputs=inputs, outputs=predictions)
    model.compile(loss=keras.losses.categorical_crossentropy,
                  optimizer=keras.optimizers.Adadelta(),
                  metrics=['accuracy'])
    return model

In [9]:
batch_size = 128
epochs = 50

model = set_up_model()
print(model.summary())

history = model.fit_generator(datagen.flow(x_train, y_train,  batch_size=batch_size),
                  epochs=epochs,
                  verbose=1,
                  validation_data=(x_test, y_test))

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 32, 32, 3)         0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 30, 30, 32)        896       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 28, 28, 32)        9248      
_________________________________________________________________
dropout_1 (Dropout)          (None, 28, 28, 32)        0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 14, 14, 32)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 12, 12, 64)        18496     
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 10, 10, 64)        36928     
__________

391/391 [==============================] - 95s 243ms/step - loss: 0.6401 - acc: 0.7788 - val_loss: 0.5608 - val_acc: 0.8120
Epoch 47/50
391/391 [==============================] - 95s 243ms/step - loss: 0.6366 - acc: 0.7798 - val_loss: 0.6024 - val_acc: 0.7980
Epoch 48/50
391/391 [==============================] - 95s 243ms/step - loss: 0.6391 - acc: 0.7824 - val_loss: 0.5587 - val_acc: 0.8112
Epoch 49/50
391/391 [==============================] - 95s 243ms/step - loss: 0.6370 - acc: 0.7842 - val_loss: 0.6143 - val_acc: 0.7918
Epoch 50/50
391/391 [==============================] - 95s 242ms/step - loss: 0.6324 - acc: 0.7841 - val_loss: 0.6663 - val_acc: 0.7831
